<a href="https://colab.research.google.com/github/Mytros/DS_ML_homework/blob/main/HW_2_2_%D0%9D%D0%B5%D0%B7%D0%B1%D0%B0%D0%BB%D0%B0%D0%BD%D1%81%D0%BE%D0%B2%D0%B0%D0%BD%D0%B0_%D0%B1%D0%B0%D0%B3%D0%B0%D1%82%D0%BE%D0%BA%D0%BB%D0%B0%D1%81%D0%BE%D0%B2%D0%B0_%D0%BA%D0%BB%D0%B0%D1%81%D0%B8%D1%84%D1%96%D0%BA%D0%B0%D1%86%D1%96%D1%8F_Dmytro_Koval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.multiclass import OneVsOneClassifier


from google.colab import drive

In [ ]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/DS_ML/customer_segmentation_train.csv', index_col = 0)

import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
missing_values = df.isnull().sum()
print("Missing values:\n", missing_values[missing_values > 0])

Missing values:
 Ever_Married       140
Graduated           78
Profession         124
Work_Experience    829
Family_Size        335
Var_1               76
dtype: int64


In [ ]:
# Cat
categorical_cols = df.select_dtypes(include='object').columns.tolist()

# Num - target 'Segmentation'
numerical_cols = df.select_dtypes(include='number').drop('Segmentation', errors='ignore').columns.tolist()

print("Cat:", categorical_cols)
print("Num:", numerical_cols)

Cat: ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1', 'Segmentation']
Num: ['Age', 'Work_Experience', 'Family_Size']


In [ ]:
for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

df[numerical_cols] = df[numerical_cols].fillna(df[numerical_cols].mean())
df[categorical_cols] = df[categorical_cols].fillna(df[categorical_cols].mode().iloc[0])

print("missing values:\n", df.isnull().sum().sum())


missing values:
 0


In [ ]:
le_target = LabelEncoder()
df['Segmentation'] = le_target.fit_transform(df['Segmentation'])

In [ ]:
X = df.drop('Segmentation', axis=1)
y = df['Segmentation']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print("Розподіл класів у y_train:\n", y_train.value_counts(normalize=True))
print("Розподіл класів у y_test:\n", y_test.value_counts(normalize=True))

X_train: (6454, 9), X_test: (1614, 9)
Розподіл класів у y_train:
 Segmentation
3    0.281066
0    0.244500
2    0.244190
1    0.230245
Name: proportion, dtype: float64
Розподіл класів у y_test:
 Segmentation
3    0.281289
0    0.244114
2    0.244114
1    0.230483
Name: proportion, dtype: float64


**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [ ]:
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes(include='object').columns.tolist()

print("Cat cols:", categorical_cols)
print("Num cols:", numeric_cols)

Cat cols: ['Gender', 'Ever_Married', 'Graduated', 'Profession', 'Spending_Score', 'Var_1']
Num cols: ['Age', 'Work_Experience', 'Family_Size']


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Pipeline
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean'))
])

cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, numeric_cols),
    ('cat', cat_pipeline, categorical_cols)
])


In [ ]:
X_train_processed = preprocessor.fit_transform(X_train)
X_train_processed.shape


(6454, 28)

In [ ]:
cat_indices = [X_train.columns.get_loc(col) for col in categorical_cols]
print("cat cols SMOTENC:", cat_indices)


cat cols SMOTENC: [0, 1, 3, 4, 6, 8]


In [ ]:
from imblearn.over_sampling import SMOTENC

smotenc = SMOTENC(
    categorical_features=cat_indices,
    random_state=42
)

X_resampled_smote, y_resampled_smote = smotenc.fit_resample(X_train.values, y_train)
print("SMOTENC: shape X_resampled:", X_resampled_smote.shape, "y_resampled:", y_resampled_smote.shape)


SMOTENC: shape X_resampled: (7256, 9) y_resampled: (7256,)


In [ ]:
from imblearn.combine import SMOTETomek

smote_tomek = SMOTETomek(
    random_state=42
)

X_resampled_smote_tomek, y_resampled_smote_tomek = smote_tomek.fit_resample(X_train_processed, y_train)
print("SMOTE-Tomek: shape X_resampled:", X_resampled_smote_tomek.shape, "y_resampled:", y_resampled_smote_tomek.shape)


SMOTE-Tomek: shape X_resampled: (6390, 28) y_resampled: (6390,)


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

pipeline_ovr_original = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', LogisticRegression(multi_class='ovr', random_state=42))
])

pipeline_ovr_original.fit(X_train, y_train)
y_pred_ovr_original = pipeline_ovr_original.predict(X_test)

print(" Original data:")
print(classification_report(y_test, y_pred_ovr_original))
print("Accuracy:", accuracy_score(y_test, y_pred_ovr_original))


 Original data:
              precision    recall  f1-score   support

           0       0.41      0.46      0.43       394
           1       0.39      0.16      0.23       372
           2       0.49      0.63      0.55       394
           3       0.65      0.76      0.70       454

    accuracy                           0.51      1614
   macro avg       0.49      0.50      0.48      1614
weighted avg       0.49      0.51      0.49      1614

Accuracy: 0.5142503097893433


In [ ]:
# SMOTENC
X_resampled_smote_processed = preprocessor.transform(pd.DataFrame(X_resampled_smote, columns=X_train.columns))

model_ovr_smote = LogisticRegression(multi_class='ovr', random_state=42)
model_ovr_smote.fit(X_resampled_smote_processed, y_resampled_smote)

y_pred_ovr_smote = model_ovr_smote.predict(preprocessor.transform(X_test))

print("SMOTENC:")
print(classification_report(y_test, y_pred_ovr_smote))
print("Accuracy:", accuracy_score(y_test, y_pred_ovr_smote))


SMOTENC:
              precision    recall  f1-score   support

           0       0.42      0.49      0.45       394
           1       0.42      0.25      0.31       372
           2       0.52      0.60      0.56       394
           3       0.67      0.72      0.69       454

    accuracy                           0.52      1614
   macro avg       0.51      0.51      0.50      1614
weighted avg       0.52      0.52      0.51      1614

Accuracy: 0.5247831474597274


In [ ]:
model_ovr_smote_tomek = LogisticRegression(multi_class='ovr', max_iter=1000, random_state=42)
model_ovr_smote_tomek.fit(X_resampled_smote_tomek, y_resampled_smote_tomek)

y_pred_ovr_smote_tomek = model_ovr_smote_tomek.predict(preprocessor.transform(X_test))

print("SMOTE-Tomek:")
print(classification_report(y_test, y_pred_ovr_smote_tomek))
print("Accuracy:", accuracy_score(y_test, y_pred_ovr_smote_tomek))


SMOTE-Tomek:
              precision    recall  f1-score   support

           0       0.43      0.47      0.45       394
           1       0.38      0.26      0.31       372
           2       0.51      0.58      0.54       394
           3       0.68      0.72      0.70       454

    accuracy                           0.52      1614
   macro avg       0.50      0.51      0.50      1614
weighted avg       0.51      0.52      0.51      1614

Accuracy: 0.5192069392812887


In [ ]:
from sklearn.metrics import f1_score

macro_f1_original = f1_score(y_test, y_pred_ovr_original, average='macro')
macro_f1_smote = f1_score(y_test, y_pred_ovr_smote, average='macro')
macro_f1_smote_tomek = f1_score(y_test, y_pred_ovr_smote_tomek, average='macro')

print("Macro-F1:")
print(f"Original: {macro_f1_original:.4f}")
print(f"SMOTENC: {macro_f1_smote:.4f}")
print(f"SMOTE-Tomek: {macro_f1_smote_tomek:.4f}")


Macro-F1:
Original: 0.4782
SMOTENC: 0.5044
SMOTE-Tomek: 0.5005


Висновок:

- Модель, натренована на даних після SMOTENC, показала найвищий macro F1-score (0.50) та найкращу accuracy (0.5248) серед трьох моделей.

- Але різниця між моделями є мінімальною. Отже, навіть збалансування класів через SMOTE або SMOTE-Tomek не дає суттєвого поліпшення. Можливо, у  моделі просто немає достатньо сильних фічей у датасеті для якісного розділення сегментів.